In [1]:
import pandas as pd

In [5]:
snowfall = pd.read_csv("snotel_simple.csv")
snowfall

,Date,Station Id,Station Name,Elevation (ft),Precipitation Accumulation (in) Start of Month Values,Snow Water Equivalent (in) Start of Month Values,Snow Depth (in) Start of Month Values,Air Temperature Average (degF),Air Temperature Maximum (degF),Air Temperature Minimum (degF)
0,Oct-94,907,Agua Canyon,8900,0.0,0.0,NaN,NaN,NaN,NaN
1,Nov-94,907,Agua Canyon,8900,2.0,0.0,NaN,NaN,NaN,NaN
2,Dec-94,907,Agua Canyon,8900,4.8,2.3,NaN,NaN,NaN,NaN
3,Jan-95,907,Agua Canyon,8900,7.5,4.7,NaN,NaN,NaN,NaN
4,Feb-95,907,Agua Canyon,8900,13.1,10.5,NaN,NaN,NaN,NaN
5,Mar-95,907,Agua Canyon,8900,16.1,12.5,NaN,NaN,NaN,NaN
6,Apr-95,907,Agua Canyon,8900,21.2,15.5,NaN,NaN,NaN,NaN
7,May-95,907,Agua Canyon,8900,23.9,5.8,NaN,NaN,NaN,NaN
8,Jun-95,907,Agua Canyon,8900,25.8,0.0,NaN,NaN,NaN,NaN
9,Jul-95,907,Agua Canyon,8900,27.0,0.0,NaN,NaN,NaN,NaN


In [4]:
sites = pd.read_csv("snotel_sites.csv")
sites

,# Data for the following site(s) are contained in this file:,Unnamed: 1
0,#,NaN
1,#\tSNOTEL 907: Agua Canyon,UT
2,#\tSNOTEL 992: Bear River RS,UT
3,#\tSNOTEL 329: Beaver Dams,UT
4,#\tSNOTEL 330: Beaver Divide,UT
5,#\tSNOTEL 332: Ben Lomond Peak,UT
6,#\tSNOTEL 333: Ben Lomond Trail,UT
7,#\tSNOTEL 1214: Bevans Cabin,UT
8,#\tSNOTEL 339: Big Flat,UT
9,#\tSNOTEL 348: Black Flat-U.M. Ck,UT
